# Create Test CIFAR10 Dataset for Kafka

 * Create a set of tensorflow REST requests
 * Create a set of tensorflow proto requests
    * The binary proto file will begin each request with the number of bytes in a 4-byte field as proto messages are not self delimiting.
    

In [34]:
import tensorflow as tf
from tensorflow_serving.apis import prediction_service_pb2_grpc, predict_pb2
import grpc
import json

Load CIFAR10 Data

In [33]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = y_train.astype('int64').reshape(-1,)
y_test = y_test.astype('int64').reshape(-1,)

Save Tensorflow REST requests into a file

In [35]:
with open("cifar10_tensorflow.json","w") as f:
    for idx in range(0,X_train.shape[0]):
        row = X_train[idx:idx+1]
        data = {"instances":row.tolist()}
        dataStr = json.dumps(data) + "\n"
        f.write(dataStr)
    f.close()

Save Tensorflow PredictRequest protos to a file.

In [37]:
with open("cifar10_tensorflow.proto","wb") as f:
    for idx in range(0,X_train.shape[0]):
        row = X_train[idx:idx+1]
        request = predict_pb2.PredictRequest()
        request.model_spec.name = "resnet32"
        request.model_spec.signature_name = "serving_default"
        request.inputs["input_1"].CopyFrom(tf.make_tensor_proto(X_train[idx:idx+1]))
        dataStr = request.SerializeToString()
        l = len(dataStr)
        lb = l.to_bytes(4, byteorder='big', signed=True)
        f.write(lb)
        f.write(dataStr)
    f.close()